## The Challenge

We have a research analyst team who provide recommendations to our portfolio managers. Unfortunately not all of these recommendations lead to a positive outcome. We need a better way to stress test the recommendations of our analyst team in a live environment to ensure that they are robust before they are passed onto our portfolio managers.

We also need to understand what information an analyst was presented with when they made a recommendation. This will allow us to understand the reasoning behind the decision at the time that it was made. This has been very difficult for us to determine previously.

## The Solution

Using LUSID we can construct a "paper portfolio" for each member of our analyst team. Using this paper portfolio they will be able to make trades based on their own recommendations to see how they would have performed under close to real market conditions. We can do this by:

1) Creating a logically separate environment for each analyst

2) Creating our tradeable instrument universe for each analyst

3) Creating a transaction portfolio for each analyst to hold their tradeable instrument universe and creating a reference portfolio for each analyst which contains the index of their tradeable instrument universe and can be used as a benchmark for performance

4) Populating each analyst's portfolio with a starting cash balance

5) Populating each analyst's reference portfolio with the weights for each constituent of their index

6) Allowing our analysts to trade across their tradeable instrument universe and add their transactions to their transaction portfolio

7) Creating an analytics store to hold the latest prices of our tradeable instrument universe

8) Valuing our analyst's portfolio using our analytics store

9) Creating an index from our reference portfolio and building a benchmark for our analyst's to compare their performance against

*To initialise our LUSID environment run the cell below*

In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('LUSID API Version: ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.5.2933.0


![Scopes](img/paper-lusid.gif)

## 1) Creating a logically separated environment for each analyst

In LUSID we can create separate environments for each of our analysts using scopes. A scope is a container for LUSID objects and can be thought of as a separate identity namespace. Using LUSID's entitlements engine we can ensure that an analyst can only interact with objects inside their scope. Furthermore no other analyst or other stakeholder inside the organisation can access this scope unless explicitly given permission. 

For this example we will create a scope for a single analyst. 

*Run the cell below to initialise our scope. Note that the scope will have a unique 4 character code appended to it to make the name more unique.*

In [2]:
# Fetch our scopes
scope_id = import_data.create_scope_id()
analyst_scope_code = 'analyst-paper-{}'.format(scope_id)

prettyprint.heading('Analyst Scope Code', analyst_scope_code)

Analyst Scope Code: analyst-paper-3794-d015-5bc6-f4


*Our LUSID environment now looks like the below. We will update this with each step.*

![Scopes](img/paper-scopes.gif)

## 2) Creating our tradeable instrument universe for each analyst

Before we can set-up our transaction and reference portfolios we need to define the tradeable instrument universe for our analyst. In this case let us assume that the analyst has been tasked with making recommendations to buy or sell equities from the London Stock Exchange FTSE techMark Focus index.

*Run the cell below to import our instrument universe*

In [3]:
# Import our instrument universe from a CSV file using Pandas
instrument_universe = pd.read_csv('data/paper-instruments.csv')
# Look at the first 10 instruments
instrument_universe.head(n=10)

,instrument_name,currency,figi,ticker,isin,sedol,client_internal
0,AVEVA GRP,GBP,BBG000C21Y87,AVV,GB00BBG9VN75,BBG9VN7,imd_56344535
1,BAE SYS.,GBP,BBG000BD5TW4,BA.,GB0002634946,263494,imd_56344983
2,BATM ADVANCED,GBP,BBG000BFJD77,BVC,IL0010849045,911146,imd_56344098
3,BTG,GBP,BBG000BZZK79,BTG,GB0001001592,100159,imd_56344345
4,COBHAM,GBP,BBG000BS6810,COB,GB00B07KD360,B07KD36,imd_56344363
5,COMPUTACENTER,GBP,BBG000BN7CL9,CCC,GB00BV9FP302,BV9FP30,imd_56344296
6,CONSORT MED.,GBP,BBG000BD8278,CSRT,GB0000946276,94627,imd_56344187
7,DIALIGHT,GBP,BBG000BLQNG4,DIA,GB0033057794,3305779,imd_56344572
8,GENUS,GBP,BBG000CTMKX7,GNS,GB0002074580,207458,imd_56344789
9,GRESHAM TECH,GBP,BBG000BS2592,GHT,GB0008808825,880882,imd_56344132


Now that we have our instruments, we can upsert our instruments in a batch. You can read more about upserting instruments in the LUSID API documentation - [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments)

*Run the cell below to upsert our instruments into LUSID*

In [4]:
# Initialise our batch upsert request
batch_upsert_request = {}
# Iterate over our instrument universe
for row, instrument in instrument_universe.iterrows():

    # Set our identifier columns
    identifier_columns = [
            ('isin', 'Isin'), 
            ('figi', 'Figi'), 
            ('ticker', 'Ticker'),
            ('sedol', 'Sedol'),
            ('client_internal', 'ClientInternal')
    ]
    
    # Create our identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier[1]] = models.InstrumentIdValue(
            value=instrument[identifier[0]])
        
    # Add the instrument to our batch request using the FIGI as the main unique identifier
    batch_upsert_request[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers=identifiers)
                                                                           
# Call LUSID to upsert our batch
instrument_response = client.instruments.upsert_instruments(
    requests=batch_upsert_request)

# Pretty print the response from LUSID
prettyprint.instrument_response(instrument_response, identifier='Figi')

Instrument Successfully Upserted: BAE SYS.
Figi ID: BBG000BD5TW4
LUSID Instrument ID: LUID_M7GC3JM0


Instrument Successfully Upserted: SPECTRIS
Figi ID: BBG000CNV8Z5
LUSID Instrument ID: LUID_1XZGIGBC


Instrument Successfully Upserted: MEGGITT
Figi ID: BBG000BDT1M0
LUSID Instrument ID: LUID_L1SDRWO3


Instrument Successfully Upserted: COMPUTACENTER
Figi ID: BBG000BN7CL9
LUSID Instrument ID: LUID_PQZJTKML


Instrument Successfully Upserted: GENUS
Figi ID: BBG000CTMKX7
LUSID Instrument ID: LUID_3RXQV5ZC


Instrument Successfully Upserted: MICRO FOCUS
Figi ID: BBG000G4KJC6
LUSID Instrument ID: LUID_GJZI0FMZ


Instrument Successfully Upserted: RICARDO
Figi ID: BBG000BDYQS3
LUSID Instrument ID: LUID_QE1MINVX


Instrument Successfully Upserted: BATM ADVANCED
Figi ID: BBG000BFJD77
LUSID Instrument ID: LUID_E7V0TK06


Instrument Successfully Upserted: SAGE GRP.
Figi ID: BBG000BN0PP3
LUSID Instrument ID: LUID_WY858B7W


Instrument Successfully Upserted: BTG
Figi ID: BBG000BZZK79
LUSID Instrum

Our LUSID environment now has an instrument master containing all of our instruments that we will be trading.

![Scopes](img/paper-instrumentmaster.gif)

During the upsertion process you may have noticed that that every instrument that is created in LUSID given a unique LUSID Instrument Id or LUID for short. This ID is used for many methods and is how LUSID uniquely identifies an instrument.

Note that in addition to our instruments, there is also cash in our holdings. Instruments that represent cash are already set-up in LUSID by default. In LUSID the identifier for cash is the ISO4217 currency code. For example British Pounds have an identifier of GBP. 

## 3) Creating a transactions portfolio for each analyst to hold their tradeable instrument universe and creating a reference portfolo for each analyst which contains the index of their tradeable instrument universe and can be used as a benchmark for performance

Now that we have populated our tradeable instrument universe, we need to create our analyst's transaction portfolio for making trades and their reference portfolio for tracking the index. Each portfolio can be referenced by its unique 'code'. Let us define the code for our two portfolios.

*Run the cell below to define our portfolio codes*

In [5]:
# Define unique codes for each of our portfolios
transaction_portfolio_code = 'LSE_FTSE_techMarkFocus_transactions'
reference_portfolio_code = 'LSE_FTSE_techMarkFocus_reference'
# Pretty print our codes
prettyprint.heading('Transaction Portfolio Code', transaction_portfolio_code)
prettyprint.heading('Reference Portfolio Code', reference_portfolio_code)

Transaction Portfolio Code: LSE_FTSE_techMarkFocus_transactions
Reference Portfolio Code: LSE_FTSE_techMarkFocus_reference


Now that we have the code for our portfolios we can go ahead and create them in LUSID. Let us first create our transaction portfolio. Note that we will create both portfolios as being available as of 3 days ago. This will allow us to add backdated transactions and compute backdated valuations.

You can read more about creating transaction portfolios in the LUSID API documentation: [LUSID API Docs: Creating Transaction Portfolios](https://docs.lusid.com/#operation/CreatePortfolio)

*Run the cell below to create our transaction portfolio*

In [6]:
# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3))

# Create the request to add our portfolio
transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=transaction_portfolio_code,
    code=transaction_portfolio_code,
    base_currency='GBP',
    description='Paper transaction portfolio',
    created=portfolio_creation_date)

# Call LUSID to create our portfolio
portfolio_response = client.transaction_portfolios.create_portfolio(
    scope=analyst_scope_code,
    create_request=transaction_portfolio_request)

# Pretty print the response from LUSID
prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: analyst-paper-3794-d015-5bc6-f4
Code: LSE_FTSE_techMarkFocus_transactions
Portfolio Effective From: 2019-07-27 08:15:27.712071+00:00
Portfolio Created On: 2019-07-30 08:15:27.884903+00:00



![Scopes](img/paper-transactionsportfolio.gif)

Now that we have created our transaction portfolio we can also create our referene portfolio to track the techMarkFocus index. 

You can read more about creating reference portfolios in the LUSID API documentation: [LUSID API Docs: Creating Reference Portfolios](https://docs.lusid.com/#operation/CreateReferencePortfolio)

*Run the cell below to create our reference portfolio*

In [7]:
# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3))

# Create the request to add our portfolio
reference_portfolio_request = models.CreateReferencePortfolioRequest(
    display_name=reference_portfolio_code,
    code=reference_portfolio_code,
    description='Paper reference portfolio',
    created=portfolio_creation_date)

# Call LUSID to create our reference portfolio
portfolio_response = client.reference_portfolios.create_reference_portfolio(
    scope=analyst_scope_code,
    reference_portfolio=reference_portfolio_request)

# Pretty print our response from LUSID
prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: analyst-paper-3794-d015-5bc6-f4
Code: LSE_FTSE_techMarkFocus_reference
Portfolio Effective From: 2019-07-27 08:15:30.614433+00:00
Portfolio Created On: 2019-07-30 08:15:30.847838+00:00



Our LUSID environment is now populated with our two portfolios. One for our analyst to make trades across and the other to track the weights of the index of the tradeable instrument universe to produce a benchmark.

![Scopes](img/paper-referenceportfolio.gif)

## 4) Populating our analyst's portfolio with a starting cash balance

To set our starting cash balance we use LUSID's 'Set Holdings' capability. This allows us to set the holdings of a portfolio to a given state. In this case we will add in a starting cash balance. Let us give the analyst a starting cash balance of £30,000,000. We will make this balance effective from just after the creation of our portfolio. 

*Run the cell below to set our starting cash balance*

In [8]:
# Set the date from which the cash balance will apply to be just after portfolio creation
holdings_effective_date = (datetime.now(pytz.UTC) - timedelta(days=2.9))
# Define our initial cash balance
initial_cash_balance = 30000000

# Create a holding adjustment to set our initial cash balance
holding_adjustment = [
    models.AdjustHoldingRequest(
        instrument_identifiers={
            'Instrument/default/Currency': 'GBP'},
        tax_lots=[
            models.TargetTaxLotRequest(
                units=initial_cash_balance,
                cost=models.CurrencyAndAmount(
                    amount=initial_cash_balance,
                    currency='GBP'),
                portfolio_cost=initial_cash_balance,
                price=1)
                ]
    )
]

# Call LUSID to set our initial cash balance
set_holdings_response = client.transaction_portfolios.set_holdings(
    scope=analyst_scope_code,
    code=transaction_portfolio_code,
    effective_at=holdings_effective_date,
    holding_adjustments=holding_adjustment)

# Pretty print our response from LUSID
prettyprint.set_holdings_response(
    set_holdings_response, 
    analyst_scope_code, 
    transaction_portfolio_code)

Holdings Successfully Set for Portfolio
Scope: analyst-paper-3794-d015-5bc6-f4
Code: LSE_FTSE_techMarkFocus_transactions
Holdings Effective From: 2019-07-27 10:39:33.549067+00:00
Holdings Created On: 2019-07-30 08:15:33.834980+00:00



Our transactions portfolio is now populated with an initial cash balance and is no longer empty.

![Scopes](img/paper-transactionsportfoliofilled.gif)

## 5) Populating our reference portfolio with the correct weights so that it tracks the index as closely as possible

To set up our reference portfolio we need to initialise it with the constituents that make up the index as well as their weight. In this case we are going to weight the consitutents based on their market capitalisation. We will import their market capitalisation from a CSV file generated from one of our internal systems. We will also configure our reference portfolio to be updated once a quarter. We could update this more frequently if we wished. 

*Run the cell below to import our market capitalisation figures for our instruments*

In [9]:
# Import the market capitalisation of each of our instruments in the index
instrument_market_cap = pd.read_csv('data/paper-weights.csv')
# Take a look at the first 10 market capitalisations
instrument_market_cap.head(n=10)

,instrument_name,currency,figi,ticker,isin,sedol,marketcap
0,AVEVA GRP,GBP,BBG000C21Y87,AVV,GB00BBG9VN75,BBG9VN7,4502.68
1,BAE SYS.,GBP,BBG000BD5TW4,BA.,GB0002634946,0263494,16349.66
2,BATM ADVANCED,GBP,BBG000BFJD77,BVC,IL0010849045,911146,185.25
3,BTG,GBP,BBG000BZZK79,BTG,GB0001001592,100159,3202.20
4,COBHAM,GBP,BBG000BS6810,COB,GB00B07KD360,B07KD36,2579.90
5,COMPUTACENTER,GBP,BBG000BN7CL9,CCC,GB00BV9FP302,BV9FP30,1161.96
6,CONSORT MED.,GBP,BBG000BD8278,CSRT,GB0000946276,0094627,422.56
7,DIALIGHT,GBP,BBG000BLQNG4,DIA,GB0033057794,3305779,133.39
8,GENUS,GBP,BBG000CTMKX7,GNS,GB0002074580,0207458,1455.72
9,GRESHAM TECH,GBP,BBG000BS2592,GHT,GB0008808825,0880882,64.34


Now that we have our market capitalisation figures we can upsert our constituents into the reference portfolio. We can add the weights in any format and LUSID will normalise them to equal 1. 

You can read more about upserting constituents in the LUSID API documentation: [LUSID API Docs: Upserting Constituents to a Reference Portfolio](https://docs.lusid.com/#operation/UpsertReferencePortfolioConstituents)

*Run the cell below to upsert our constituents*

In [10]:
# Initialise a list to hold our constituents
constituents = []
# Work out the total market capitalisation of the entire index
total = instrument_market_cap['marketcap'].sum()

# Iterate over instrument unvierse to add each constituent to our list
for row in instrument_market_cap.iterrows():
    # Collect our instrument
    instrument = row[1]
    # Calculate our constituents weight based on market cap and add it to our list
    constituents.append(models.ReferencePortfolioConstituentRequest(
        instrument_identifiers={
            'Instrument/default/Figi': instrument['figi']},
        weight=instrument['marketcap']/total,
        currency=instrument['currency']))

# Create our request to add our constituents
constituents_request = models.UpsertReferencePortfolioConstituentsRequest(
    effective_from=holdings_effective_date,
    weight_type="Periodical",
    period_type="Quarterly",
    period_count=4,
    constituents=constituents)

# Call LUSID to upsert our constituents into our reference portfolio
response = client.reference_portfolios.upsert_reference_portfolio_constituents(
    scope=analyst_scope_code,
    code=reference_portfolio_code,
    constituents=constituents_request)

print ('Constituents Upserted')

Constituents Upserted


Our reference portfolio is no longer empty and now contains the weights for each instrument in our index against which the analyst's performance will be compared.

![Scopes](img/paper-referenceportfoliofilled.gif)

## 6) Allow our analysts to trade across their tradeable instrument universe and add their transactions to their transaction portfolio

Now that we have set up our portfolios we can add in our analyst's trades. These would likel be populated from an order management or similar system. In this case we are going to import the trades over the last two days from a CSV file.

*Run the cell below to import our transactions*

In [11]:
# Import our analyst's trades from a CSV file
analyst_transactions = import_data.fetch_client_transactions(
    'paper-transactions.csv',
    days_back=2)

# Pretty print their activity
analyst_transactions.head(n=20)

,transaction_id,type,portfolio,instrument_name,instrument_uid,transaction_date,settlement_date,units,transaction_price,transaction_currency,total_cost,strategy,description
0,tid_124398219481,Buy,LSE_FTSE_techMarkFocus_transactions,AVEVA GRP,BBG000C21Y87,2019-07-29T10:53:03.317488+00:00,2019-07-31T10:53:03.317488+00:00,265600,28.940000,GBP,7.686464e+06,quantitativeSignal,Purchase
1,tid_339423984894,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2019-07-29T12:10:20.808224+00:00,2019-07-31T12:10:20.808224+00:00,265600,0.005788,GBP,1.537293e+03,quantitativeSignal,Brokerage Fees
2,tid_339423984896,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2019-07-29T13:30:39.361388+00:00,2019-07-31T13:30:39.361388+00:00,265600,0.014470,GBP,3.843232e+03,quantitativeSignal,Stamp Duty
3,tid_359231290933,Buy,LSE_FTSE_techMarkFocus_transactions,BATM ADVANCED,BBG000BFJD77,2019-07-29T16:44:22.953422+00:00,2019-07-31T16:44:22.953422+00:00,750000,0.463000,GBP,3.472500e+05,fundamentalAnalysis,Purchase
4,tid_223239424244,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2019-07-29T09:05:08.518044+00:00,2019-07-31T09:05:08.518044+00:00,750000,0.000093,GBP,6.945000e+01,fundamentalAnalysis,Brokerage Fees
5,tid_354242442211,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2019-07-28T16:42:04.981414+00:00,2019-07-30T16:42:04.981414+00:00,750000,0.000231,GBP,1.736250e+02,fundamentalAnalysis,Stamp Duty
6,tid_342398989895,SellShort,LSE_FTSE_techMarkFocus_transactions,COMPUTACENTER,BBG000BN7CL9,2019-07-29T13:16:57.614575+00:00,2019-07-31T13:16:57.614575+00:00,150000,10.500000,GBP,1.575000e+06,fundamentalAnalysis,Purchase
7,tid_384809092344,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2019-07-29T14:01:45.173840+00:00,2019-07-31T14:01:45.173840+00:00,150000,0.002100,GBP,3.150000e+02,fundamentalAnalysis,Brokerage Fees
8,tid_348234938244,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2019-07-29T11:46:38.660644+00:00,2019-07-31T11:46:38.660644+00:00,150000,0.005250,GBP,7.875000e+02,fundamentalAnalysis,Stamp Duty
9,tid_239394829484,Buy,LSE_FTSE_techMarkFocus_transactions,DIALIGHT,BBG000BLQNG4,2019-07-29T12:46:39.415011+00:00,2019-07-31T12:46:39.415011+00:00,1500000,4.180000,GBP,6.270000e+06,quantitativeSignal,Purchase


We can set custom properties on our transactions. In this case we are going to create a strategy property which allows the analyst to note what strategy they were following when they made the trade. These will help us justify their trades and recommendations when it comes to making them for real. 

You can read more about creating properties in the LUSID API documentation: [LUSID API Docs: Defining a new Property](https://docs.lusid.com/#operation/CreatePropertyDefinition)

*Run the cell below to create our transaction strategy property*

In [12]:
# Create a request to define our strategy property
property_request = models.CreatePropertyDefinitionRequest(
    domain='Transaction',
    scope=analyst_scope_code,
    code='strategy',
    value_required=False,
    display_name='strategy',
    data_type_id=models.ResourceId(
        scope='system',
        code='string')
)

# Call LUSID to create our new property
property_response = client.property_definitions.create_property_definition(
    definition=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
strategy_property_key = property_response.key

# Pretty print our strategy property key
prettyprint.heading('Strategy Property Key: ', strategy_property_key)

Strategy Property Key: : Transaction/analyst-paper-3794-d015-5bc6-f4/strategy


Now that we have our trades imported and our property created we can upsert them into LUSID. Read more about adding transactions in the LUSID API documentation: [LUSID API Docs: Upserting Transactions](https://docs.lusid.com/#operation/UpsertTransactions)

*Run the cell below to upsert our transactions into our transactions portfolio*

In [14]:
# Initialise a list to hold our transactions
batch_transaction_requests = []

# Iterate over the transactions for each portfolio
for index, transaction in analyst_transactions.iterrows():
    
    if 'Cash' in transaction['instrument_name']:
        identifier_key = 'Instrument/default/Currency'
    else:
        identifier_key = 'Instrument/default/Figi'
    
    batch_transaction_requests.append(
        models.TransactionRequest(
            transaction_id=transaction['transaction_id'],
            type=transaction['type'],
            instrument_identifiers={
                identifier_key: transaction['instrument_uid']},
            transaction_date=transaction['transaction_date'],
            settlement_date=transaction['settlement_date'],
            units=transaction['units'],
            transaction_price=models.TransactionPrice(
                price=transaction['transaction_price'],
                type='Price'),
            total_consideration=models.CurrencyAndAmount(
                amount=transaction['total_cost'],
                currency=transaction['transaction_currency']),
            source='Client',
            transaction_currency=transaction['transaction_currency'],
            properties={
                strategy_property_key:
                    models.PerpetualProperty(
                        key=strategy_property_key,
                        value=models.PropertyValue(label_value=transaction['strategy']) 
                    )
            }
        )
    )

# Call LUSID to upsert our transactions
transaction_response = client.transaction_portfolios.upsert_transactions(
    scope=analyst_scope_code,
    code=transaction_portfolio_code,
    transactions=batch_transaction_requests)

# Pretty print the response from LUSID
prettyprint.transactions_response(
    transaction_response,
    analyst_scope_code, 
    transaction_portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: analyst-paper-3794-d015-5bc6-f4
Code: LSE_FTSE_techMarkFocus_transactions
Transactions Effective From: 2019-07-29 16:44:22.953422+00:00
Transactions Created On: 2019-07-30 08:17:03.963561+00:00



Our transaction portfolio is now populated with our analyst's paper trades.

## 7) Create an analytics store to hold the latest prices of our tradeable instrument universe

To value our portfolios we need current prices. We store these prices in an analytics store. There are a number of different ways and sources that we can get our pricing information. In this case we are going to load the prices when we set up our portfolios and the current prices from a CSV file.

*Run the cell below to import our pricing information*

In [15]:
# Import our instrument prices from a CSV file
instrument_prices = pd.read_csv('data/paper-analytics.csv')
# Pretty print our pricing
instrument_prices.head(n=10)

,instrument_name,currency,figi,price_original,price_current,ticker,isin,sedol
0,AVEVA GRP,GBP,BBG000C21Y87,27.8400,30.317760,AVV,GB00BBG9VN75,BBG9VN7
1,BAE SYS.,GBP,BBG000BD5TW4,5.1540,4.782912,BA.,GB0002634946,263494
2,BATM ADVANCED,GBP,BBG000BFJD77,0.4590,0.424575,BVC,IL0010849045,911146
3,BTG,GBP,BBG000BZZK79,8.2750,8.970100,BTG,GB0001001592,100159
4,COBHAM,GBP,BBG000BS6810,1.0715,1.022211,COB,GB00B07KD360,B07KD36
5,COMPUTACENTER,GBP,BBG000BN7CL9,10.6800,11.459640,CCC,GB00BV9FP302,BV9FP30
6,CONSORT MED.,GBP,BBG000BD8278,8.7000,9.309000,CSRT,GB0000946276,94627
7,DIALIGHT,GBP,BBG000BLQNG4,3.9200,3.551520,DIA,GB0033057794,3305779
8,GENUS,GBP,BBG000CTMKX7,22.4600,20.595820,GNS,GB0002074580,207458
9,GRESHAM TECH,GBP,BBG000BS2592,0.9450,1.000755,GHT,GB0008808825,880882


Now that we have our pricing information we can store this data inside LUSID in an analytics store. We will create two stores one for the prices from 3 days ago when we originally set-up the portfolios and one for the current prices.

You can read more abouy creating and setting analytics stores in the LUSID API documentation: [LUSID API Docs: Creating an Analytics Store](https://docs.lusid.com/#operation/CreateAnalyticStore) & [LUSID API Docs: Setting Analytic Data](https://docs.lusid.com/#operation/SetAnalytics)

*Run the cell below to set-up our analytics stores and set our analytic data*

In [16]:
# Set our analytics effective dates
analytics_effective_date = datetime.now(pytz.UTC) - timedelta(days=3)
today = datetime.now(pytz.UTC)
analytics_store_dates = [analytics_effective_date, today]

# Create prices via instrument, analytic
instrument_analytics_inception = []
instrument_analytics_today = []

# Create dummy prices of 1 for all instruments except cash
for row in instrument_prices.iterrows():
    
    instrument = row[1]
    
    if 'Cash' in instrument['instrument_name']:
        continue
        
    # Get our Lusid Instrument Id
    luid = client.instruments.get_instrument(
        identifier_type='Figi',
        identifier=instrument['figi']).lusid_instrument_id
    
    instrument_analytics_inception.append(models.InstrumentAnalytic(
        instrument_uid=luid,
        value=instrument['price_original']))
    
    instrument_analytics_today.append(models.InstrumentAnalytic(
        instrument_uid=luid,
        value=instrument['price_current']))
    
instrument_analytics = [instrument_analytics_inception, instrument_analytics_today]
    
for date, instrument_analytic_set in zip(analytics_store_dates, instrument_analytics):
    
    # Create analytics store request
    analytics_store_request = models.CreateAnalyticStoreRequest(
        scope=analyst_scope_code,
        date=date.isoformat())

    # Call LUSID to create our analytics store
    client.analytics_stores.create_analytic_store(
        request=analytics_store_request)

    # Call LUSID to set up our newly created analytics store with our prices
    client.analytics_stores.set_analytics(
        scope=analyst_scope_code,
        year=date.year,
        month=date.month,
        day=date.day,
        data=instrument_analytic_set)
    
print ('Analytics Set')

Analytics Set


Our LUSID environment now looks like the below. We have a reference portfolio to hold our index which will act as a benchmark, we have a transaction portfolio for our analyst to make paper trades and we have an analytics store to value our portfolios.

![Scopes](img/paper-analyticsstore.gif)

## 8) Value each portfolio using our analytics store

We can now value our portfolios to see if the analyst has been making good decisions. Let us start by valuing our transaction portfolio to find out what our absolute return is across our tradeable instrument universe. 

We do this by making an aggregation request to LUSID. We will aggregate by instrument against the number of units, cost, present value and price. 

You can read more about aggregation in the LUSID API documentation: [LUSID API Docs: Aggregate Data in a Portfolio](https://docs.lusid.com/#operation/GetAggregationByPortfolio)

*Run the cell below to make our aggregation request on our transaction portfolio and see how our analyst is performing*

In [17]:
# Create our aggregation request
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=analyst_scope_code,
        code='default'),
    effective_at=today,
    metrics=[
        models.AggregateSpec(
            key='Holding/default/SubHoldingKey',
            op='Value'),
        models.AggregateSpec(
            key='Holding/default/Units',
            op='Sum'),
        models.AggregateSpec(
            key='Holding/default/Cost',
            op='Sum'),
        models.AggregateSpec(
            key='Holding/default/PV',
            op='Sum'),
       models.AggregateSpec(
           key='Holding/default/Price',
           op='Sum') 
    ],
group_by=[
    'Holding/default/SubHoldingKey'
])
                                                        
# Call LUSID to aggregate across all of our portfolios
aggregated_portfolio = client.aggregation.get_aggregation_by_portfolio(
    scope=analyst_scope_code,
    code=transaction_portfolio_code,
    request=aggregation_request)

prettyprint.aggregation_response_paper(aggregated_portfolio)

Instrument :LusidInstrumentId=LUID_HZ3NON8M/GBP
Units :55000.0
Current Price :£5.6
Present Value :£308154.0
Cost :£305250.0
Return :0.9514%

Instrument :LusidInstrumentId=LUID_3AU2OY1O/GBP
Units :98000.0
Current Price :£19.51
Present Value :£1912019.2
Cost :£2156000.0
Return :-11.3164%

Instrument :LusidInstrumentId=LUID_3RXQV5ZC/GBP
Units :345000.0
Current Price :£20.6
Present Value :£7105557.9
Cost :£7590000.0
Return :-6.3826%

Instrument :LusidInstrumentId=LUID_DMXWOM7H/GBP
Units :120000.0
Current Price :£13.36
Present Value :£1603546.8
Cost :£1582800.0
Return :1.3108%

Instrument :LusidInstrumentId=LUID_GJZI0FMZ/GBP
Units :-543000.0
Current Price :£13.82
Present Value :£-7503461.79
Cost :£-7384800.0
Return :-1.6068%

Instrument :LusidInstrumentId=LUID_WY858B7W/GBP
Units :-98000.0
Current Price :£6.57
Present Value :£-643432.72
Cost :£-577220.0
Return :-11.471%

Instrument :LusidInstrumentId=LUID_O58XT9J2/GBP
Units :265600.0
Current Price :£30.32
Present Value :£8052397.06
Cost :£76

## 9) Compare performance across the analyst's reference and transaction portfolios

Now that we have got our analyst's absolute return we want to see how this compares to their benchmark. In LUSID we can't value our reference portfolio directly as all it contains is weights. What we can is decide on an arbitary index level (in this case we will start with 1000), create a new transaction portfolio to hold our index and then populate it based on the weights from our reference portfolio.

First things first, let us give our index portfolio a code.

*Run the cell below to create a unique code for our index portfolio*

In [18]:
index_portfolio_code = 'LSE_FTSE_techMarkFocus_index'
prettyprint.heading('Index Portfolio Code', index_portfolio_code)

Index Portfolio Code: LSE_FTSE_techMarkFocus_index


Now we can create our index portfolio

*Run the cell below to create our index portfolio*

In [19]:
# Create the request to add our portfolio
transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=index_portfolio_code,
    code=index_portfolio_code,
    base_currency='GBP',
    description='Index transaction portfolio',
    created=portfolio_creation_date)

# Create our portfolio
portfolio_response = client.transaction_portfolios.create_portfolio(
    scope=analyst_scope_code,
    create_request=transaction_portfolio_request)

prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: analyst-paper-3794-d015-5bc6-f4
Code: LSE_FTSE_techMarkFocus_index
Portfolio Effective From: 2019-07-27 08:15:30.614433+00:00
Portfolio Created On: 2019-07-30 08:17:24.447054+00:00



We now have an index portfolio that we can use to hold our index and obtain our benchmark.

![Scopes](img/paper-indexportfolio.gif)

In [20]:
# Set an arbitary index level to start our index with
index_level = 1000
# Call LUSID - get the constituents of our index from our reference portfolio
constituents = client.reference_portfolios.get_reference_portfolio_constituents(
    scope=analyst_scope_code,
    code=reference_portfolio_code,
    effective_at=datetime.now(pytz.UTC))
# Initialise our list to hold the adjustments we need to make to our index to set it up
index_setup = []
# Get our weights from the constituents into a better format to work with
weights = {constituent.instrument_uid:constituent.weight for constituent in constituents.constituents}

# Iterate over our pricing analytics
for index, instrument in instrument_prices.iterrows():
    
    # Get our Lusid Instrument ID
    Luid = client.instruments.get_instrument(
        identifier_type='Figi',
        identifier=instrument['figi']).lusid_instrument_id
    # Get the initial price for each constituent of the index from our analytics store
    inception_price = instrument['price_original']
    # Work out how much of the index this constituent should make up using its w
    index_cost = weights[Luid] * index_level
    # Work out how many units we should therefore buy
    index_units = index_cost / inception_price
    # Create our request for this instrument 
    index_setup.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                'Instrument/default/Figi': instrument['figi']},
            tax_lots=[
                models.TargetTaxLotRequest(
                units=index_units,
                cost=models.CurrencyAndAmount(
                    amount=index_cost,
                    currency='GBP'),
                portfolio_cost=index_cost,
                price=inception_price)
            ]
        )
    )
# Call LUSID to set all of our holdings to the initial index level
adjust_holdings_response = client.transaction_portfolios.set_holdings(
    scope=analyst_scope_code,
    code=index_portfolio_code,
    effective_at=portfolio_creation_date,
    holding_adjustments=index_setup)

prettyprint.set_holdings_response(
    adjust_holdings_response, 
    analyst_scope_code, 
    index_portfolio_code)

Holdings Successfully Set for Portfolio
Scope: analyst-paper-3794-d015-5bc6-f4
Code: LSE_FTSE_techMarkFocus_index
Holdings Effective From: 2019-07-27 08:15:30.614433+00:00
Holdings Created On: 2019-07-30 08:17:30.965431+00:00



We now have an index portfolio that we can value.

![Scopes](img/paper-indexportfoliofilled.gif)

In [21]:
# Create our aggregation request
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=analyst_scope_code,
        code='default'),
        effective_at=today,
        metrics=[
            models.AggregateSpec(
                key='Holding/default/PV',
                op='Sum'),
            models.AggregateSpec(
                key='Holding/default/Cost',
                op='Sum')
        ],
group_by=['Portfolio/default/Name'])
                                                        
# Call LUSID to aggregate across all of our portfolios
aggregated_portfolio = client.aggregation.get_aggregation_by_portfolio(
    scope=analyst_scope_code,
    code=index_portfolio_code,
    request=aggregation_request
)

# Pretty print the response from LUSID
prettyprint.aggregation_response_index(aggregated_portfolio)

Initial Index Level :999.98
Current Index Level :995.97
Return :-0.4013%
